# Converter for the Financial Filings at the Sec

In [37]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2010q4.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t")
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2010q4"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

4624794
2241960
53064
194922
24168


In [38]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+-------------------------------------------------------+--------------+---------------------------+----------+--------+-------+---------------+------------+
|    | adsh                 | tag                                                   | version      | coreg                     |    ddate |   qtrs | uom   |         value |   footnote |
|----+----------------------+-------------------------------------------------------+--------------+---------------------------+----------+--------+-------+---------------+------------|
|  0 | 0000930413-10-005253 | AcceleratedShareRepurchasesFinalPricePaidPerShare     | us-gaap/2009 | nan                       | 20100331 |      1 | pure  |  56.21        |        nan |
|  1 | 0000930413-10-005253 | AcceleratedShareRepurchasesInitialPricePaidPerShare   | us-gaap/2009 | nan                       | 20100331 |      1 | pure  |  56.05        |        nan |
|  2 | 0001193125-10-255976 | AcceleratedShareRepurchasesSettlementPay

In [39]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+--------------------------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                          | version      | plabel                                                                                                                                                   |   negating |
|----+----------------------+----------+--------+--------+---------+---------+--------------------------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+------------|
|  0 | 0000890319-10-0000

In [40]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+-----------------------------------+-------+-------------+----------+----------------------+------------+----------------------------------------+-------------------------------------+---------------------+-------------+----------+----------------------+------------+-----------------------------------------+-------------------------------------+--------------+-----------+-----------+---------------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+--------------------+---------+---------------------+
|    | adsh                 |     cik | name                              |   sic | countryba   | stprba   | cityba               | zipba      | bas1                                   | bas2                                | baph                | countryma   | stprma   | cityma               | zipma      | mas1                               

In [41]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+-------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+-------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [43]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                                     0000930413-10-005253
tag         AcceleratedShareRepurchasesFinalPricePaidPerShare
version                                          us-gaap/2009
coreg                                                     NaN
ddate                                                20100331
qtrs                                                        1
uom                                                      pure
value                                                   56.21
footnote                                                  NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [44]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

# Create a single SymbolFinancialsDto

In [49]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    sfDto.symbol = val["symbol"].to_string(index = False).upper()
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    filteredBsDfPre = dfPre[(dfPre['stmt'].str.strip() == 'BS') & (dfPre['adsh'] == submitted["adsh"])]
    filteredCfDfPre = dfPre[(dfPre['stmt'].str.strip() == 'CF') & (dfPre['adsh'] == submitted["adsh"])]
    filteredIcDfPre = dfPre[(dfPre['stmt'].str.strip() == 'IC') & (dfPre['adsh'] == submitted["adsh"])]
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]    
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myDto.info = myPre["plabel"].to_string(index = False)
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(str(result))
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("file time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec.")

Json size 1574kb. Time: 47.379ms.
file 1 exportFiles/2010q4/0000930413-10-004972.json stored in 3642.86ms.
Json size 1602kb. Time: 44.205ms.
file 2 exportFiles/2010q4/0000950123-10-090627.json stored in 5767.254ms.
Json size 1643kb. Time: 44.889ms.
file 3 exportFiles/2010q4/0001193125-10-221435.json stored in 2933.984ms.
Json size 1664kb. Time: 48.277ms.
file 4 exportFiles/2010q4/0001193125-10-221654.json stored in 4336.328ms.
Json size 1673kb. Time: 46.393ms.
file 5 exportFiles/2010q4/0001193125-10-221663.json stored in 1618.412ms.
Json size 1698kb. Time: 49.503ms.
file 6 exportFiles/2010q4/0001193125-10-222237.json stored in 3350.793ms.
Json size 1719kb. Time: 47.297ms.
file 7 exportFiles/2010q4/0001104659-10-051261.json stored in 3427.362ms.
Json size 1741kb. Time: 51.25ms.
file 8 exportFiles/2010q4/0001140361-10-040079.json stored in 3141.586ms.
Json size 1819kb. Time: 50.362ms.
file 9 exportFiles/2010q4/0001193125-10-223352.json stored in 7156.417ms.
Json size 1845kb. Time: 50.227

Json size 3771kb. Time: 114.426ms.
file 77 exportFiles/2010q4/0000930413-10-005192.json stored in 6997.964ms.
Json size 3792kb. Time: 112.084ms.
file 78 exportFiles/2010q4/0000950123-10-095072.json stored in 6451.553ms.
Json size 3808kb. Time: 113.406ms.
file 79 exportFiles/2010q4/0000950123-10-095145.json stored in 4434.156ms.
Json size 3836kb. Time: 116.428ms.
file 80 exportFiles/2010q4/0000950123-10-095279.json stored in 4236.818ms.
Json size 3854kb. Time: 118.77ms.
file 81 exportFiles/2010q4/0000950123-10-095292.json stored in 2917.121ms.
Json size 3881kb. Time: 115.398ms.
file 82 exportFiles/2010q4/0000950123-10-095293.json stored in 3822.948ms.
Json size 3909kb. Time: 120.083ms.
file 83 exportFiles/2010q4/0000950123-10-095299.json stored in 6694.494ms.
Json size 3935kb. Time: 120.5ms.
file 84 exportFiles/2010q4/0001104659-10-053238.json stored in 3594.769ms.
Json size 3959kb. Time: 117.518ms.
file 85 exportFiles/2010q4/0001144204-10-055158.json stored in 3822.821ms.
Json size 399

Json size 6320kb. Time: 189.287ms.
file 152 exportFiles/2010q4/0001104659-10-053819.json stored in 4053.723ms.
Json size 6358kb. Time: 192.85ms.
file 153 exportFiles/2010q4/0001104659-10-054017.json stored in 12850.86ms.
Json size 6381kb. Time: 186.248ms.
file 154 exportFiles/2010q4/0001140361-10-042334.json stored in 4721.836ms.
Json size 6402kb. Time: 187.311ms.
file 155 exportFiles/2010q4/0001140361-10-042408.json stored in 3490.147ms.
Json size 6430kb. Time: 181.551ms.
file 156 exportFiles/2010q4/0001144204-10-055835.json stored in 3833.9ms.
Json size 6447kb. Time: 176.373ms.
file 157 exportFiles/2010q4/0001193125-10-236923.json stored in 2619.921ms.
Json size 6472kb. Time: 177.031ms.
file 158 exportFiles/2010q4/0001193125-10-237308.json stored in 3654.955ms.
Json size 6517kb. Time: 186.586ms.
file 159 exportFiles/2010q4/0001193125-10-237367.json stored in 3685.204ms.
Json size 6542kb. Time: 182.553ms.
file 160 exportFiles/2010q4/0001193125-10-237767.json stored in 6649.187ms.
Json

Json size 8587kb. Time: 238.248ms.
file 226 exportFiles/2010q4/0001144204-10-056126.json stored in 4705.993ms.
Json size 8622kb. Time: 246.791ms.
file 227 exportFiles/2010q4/0001144204-10-056127.json stored in 7292.351ms.
Json size 8655kb. Time: 243.592ms.
file 228 exportFiles/2010q4/0001144204-10-056130.json stored in 5039.687ms.
Json size 8683kb. Time: 250.769ms.
file 229 exportFiles/2010q4/0001193125-10-238300.json stored in 3913.381ms.
Json size 8713kb. Time: 247.623ms.
file 230 exportFiles/2010q4/0001193125-10-238462.json stored in 6448.338ms.
Json size 8736kb. Time: 250.198ms.
file 231 exportFiles/2010q4/0001193125-10-238768.json stored in 5526.43ms.
Json size 8769kb. Time: 245.008ms.
file 232 exportFiles/2010q4/0001193125-10-238937.json stored in 3179.828ms.
Json size 8805kb. Time: 249.963ms.
file 233 exportFiles/2010q4/0001193125-10-238973.json stored in 3890.566ms.
Json size 8911kb. Time: 251.257ms.
file 234 exportFiles/2010q4/0001193125-10-239023.json stored in 17358.746ms.
J

Json size 10757kb. Time: 296.885ms.
file 300 exportFiles/2010q4/0001047469-10-008982.json stored in 10524.74ms.
Json size 10794kb. Time: 300.788ms.
file 301 exportFiles/2010q4/0001047469-10-008983.json stored in 4379.194ms.
Json size 10836kb. Time: 303.402ms.
file 302 exportFiles/2010q4/0001047469-10-008984.json stored in 4411.087ms.
Json size 10855kb. Time: 306.496ms.
file 303 exportFiles/2010q4/0001047469-10-008985.json stored in 3004.154ms.
Json size 10890kb. Time: 305.933ms.
file 304 exportFiles/2010q4/0001047469-10-008986.json stored in 3378.809ms.
Json size 10914kb. Time: 302.731ms.
file 305 exportFiles/2010q4/0001047469-10-008991.json stored in 4326.601ms.
Json size 10949kb. Time: 308.636ms.
file 306 exportFiles/2010q4/0001047469-10-008993.json stored in 5752.192ms.
Json size 10970kb. Time: 307.531ms.
file 307 exportFiles/2010q4/0001047469-10-009001.json stored in 3257.443ms.
Json size 11004kb. Time: 310.629ms.
file 308 exportFiles/2010q4/0001082510-10-000085.json stored in 4668

...

Json size 38382kb. Time: 1063.602ms.
file 1185 exportFiles/2010q4/0001193125-10-253893.json stored in 3879.718ms.
Json size 38400kb. Time: 1076.523ms.
file 1186 exportFiles/2010q4/0001193125-10-253900.json stored in 3693.759ms.
Json size 38440kb. Time: 1082.008ms.
file 1187 exportFiles/2010q4/0001193125-10-253911.json stored in 6036.174ms.
Json size 38470kb. Time: 1170.885ms.
file 1188 exportFiles/2010q4/0001193125-10-253933.json stored in 12862.386ms.
Json size 38503kb. Time: 1152.81ms.
file 1189 exportFiles/2010q4/0001193125-10-253944.json stored in 4840.859ms.
Json size 38533kb. Time: 1323.471ms.
file 1190 exportFiles/2010q4/0001193125-10-253961.json stored in 5327.306ms.
Json size 38559kb. Time: 1145.426ms.
file 1191 exportFiles/2010q4/0001193125-10-253972.json stored in 5706.5ms.
Json size 38592kb. Time: 1147.029ms.
file 1192 exportFiles/2010q4/0001193125-10-254033.json stored in 6859.575ms.
Json size 38592kb. Time: 1134.005ms.
file 1193 exportFiles/2010q4/0001193125-10-254051.jso

Json size 40663kb. Time: 1142.337ms.
file 1257 exportFiles/2010q4/0001193125-10-258748.json stored in 4127.976ms.
Json size 40688kb. Time: 1147.161ms.
file 1258 exportFiles/2010q4/0001193125-10-258861.json stored in 6208.773ms.
Json size 40758kb. Time: 1134.525ms.
file 1259 exportFiles/2010q4/0000950123-10-105866.json stored in 9850.407ms.
Json size 40786kb. Time: 1147.553ms.
file 1260 exportFiles/2010q4/0001046188-10-000037.json stored in 6188.094ms.
Json size 40841kb. Time: 1134.236ms.
file 1261 exportFiles/2010q4/0001193125-10-260108.json stored in 6064.493ms.
Json size 40865kb. Time: 1147.171ms.
file 1262 exportFiles/2010q4/0001193125-10-260837.json stored in 4494.049ms.
Json size 40905kb. Time: 1135.425ms.
file 1263 exportFiles/2010q4/0001047469-10-009822.json stored in 5037.872ms.
Json size 40948kb. Time: 1209.415ms.
file 1264 exportFiles/2010q4/0001193125-10-261858.json stored in 15576.762ms.
Json size 40988kb. Time: 1222.53ms.
file 1265 exportFiles/2010q4/0001193125-10-261986.j

Json size 43209kb. Time: 1304.038ms.
file 1329 exportFiles/2010q4/0000950123-10-108594.json stored in 6240.201ms.
Json size 43252kb. Time: 1203.461ms.
file 1330 exportFiles/2010q4/0000950123-10-108604.json stored in 7134.226ms.
Json size 43280kb. Time: 1213.071ms.
file 1331 exportFiles/2010q4/0000950123-10-108757.json stored in 11873.754ms.
Json size 43308kb. Time: 1206.103ms.
file 1332 exportFiles/2010q4/0000950123-10-108803.json stored in 5205.35ms.
Json size 43380kb. Time: 1205.161ms.
file 1333 exportFiles/2010q4/0000950123-10-108829.json stored in 10879.728ms.
Json size 43417kb. Time: 1221.172ms.
file 1334 exportFiles/2010q4/0001047469-10-010040.json stored in 6257.747ms.
Json size 43456kb. Time: 1212.533ms.
file 1335 exportFiles/2010q4/0001104659-10-059917.json stored in 12683.237ms.
Json size 43475kb. Time: 1235.441ms.
file 1336 exportFiles/2010q4/0001193125-10-267778.json stored in 8560.087ms.
Json size 43516kb. Time: 1203.046ms.
file 1337 exportFiles/2010q4/0001193125-10-268147

Json size 45409kb. Time: 1322.981ms.
file 1401 exportFiles/2010q4/0001140361-10-048808.json stored in 5971.962ms.
Json size 45429kb. Time: 1267.896ms.
file 1402 exportFiles/2010q4/0001144204-10-065433.json stored in 4433.73ms.
Json size 45455kb. Time: 1327.916ms.
file 1403 exportFiles/2010q4/0001166126-10-000055.json stored in 6929.318ms.
Json size 45478kb. Time: 1340.637ms.
file 1404 exportFiles/2010q4/0001168213-10-000075.json stored in 4561.212ms.
Json size 45510kb. Time: 1347.017ms.
file 1405 exportFiles/2010q4/0001185185-10-001425.json stored in 4422.225ms.
Json size 45537kb. Time: 1346.266ms.
file 1406 exportFiles/2010q4/0001193125-10-276610.json stored in 7565.156ms.
Json size 45567kb. Time: 1329.781ms.
file 1407 exportFiles/2010q4/0001193125-10-276629.json stored in 5214.298ms.
Json size 45594kb. Time: 1244.823ms.
file 1408 exportFiles/2010q4/0001206774-10-002571.json stored in 6378.297ms.
Json size 45622kb. Time: 1263.505ms.
file 1409 exportFiles/2010q4/0001206774-10-002589.js

Json size 47714kb. Time: 1387.506ms.
file 1473 exportFiles/2010q4/0001193125-10-290314.json stored in 5699.521ms.
Json size 47746kb. Time: 1393.676ms.
file 1474 exportFiles/2010q4/0001437749-10-004597.json stored in 13131.979ms.
file time 9010.025403sec.
